In [1]:
import pandas as pd
import requests
import json
import os 

foursquare_key = os.environ["FOURSQUARE_API_KEY"]
yelp_key = os.environ["YELP_API_KEY"]

In [2]:
# Read the CSV files
df = pd.read_csv('df_city_bike.csv')

In [3]:
df

,name,empty_slots,free_bikes,latitude,longitude
0,56 - CLN 110 W1 Norte,5,6,-15.758862,-47.888820
1,54 - CLN 107,4,7,-15.766000,-47.886460
2,42 - CLN 204 L1 Norte,4,7,-15.776620,-47.877490
3,48 - CLN 209 L1 Norte,9,6,-15.757370,-47.882230
4,43 - SQN 405 L2 Norte,7,8,-15.771630,-47.875050
...,...,...,...,...,...
65,70 - UNB Centro Olímpico,8,3,-15.765020,-47.858400
66,17 - Funarte,5,10,-15.791163,-47.897600
67,61 - CLN 113 W1 Norte,5,6,-15.750380,-47.891660
68,28 - Banco Central,6,5,-15.799740,-47.884110


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [8]:
import pandas as pd
import requests

def convert_and_process(df, foursquare_key):
    # Convert latitude and longitude columns to strings
    df['latitude'] = df['latitude'].astype(str)
    df['longitude'] = df['longitude'].astype(str)

    # Define the headers
    headers = {
        "accept": "application/json",
        "Authorization": foursquare_key
    }

    # Send requests and process responses
    for _, row in df.iterrows():
        latitude = row['latitude']
        longitude = row['longitude']
        url = f"https://api.foursquare.com/v3/places/search?ll={latitude}%2C{longitude}&radius=1000&fields=categories%2Crating%2Cgeocodes%2Ccategories%2Clocation%2Cname%2Cfsq_id%2Cstats%2Cprice&limit=50"
        
        response = requests.get(url, headers=headers)
        response.raise_for_status()
data_fq = response.json()

        # Process the response data as needed
        # ...


# Call the function with the DataFrame and the foursquare_key
convert_and_process(df, foursquare_key)

print(data_fq)


{'results': [{'fsq_id': '53ee2c96498ecb2bcceb5fba', 'categories': [{'id': 13065, 'name': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}}], 'geocodes': {'drop_off': {'latitude': -15.790526, 'longitude': -47.880904}, 'main': {'latitude': -15.790561, 'longitude': -47.880681}, 'roof': {'latitude': -15.790561, 'longitude': -47.880681}}, 'location': {'address': 'St. Bancario Norte 1 - Asa Norte', 'address_extended': 'Bloco B', 'country': 'BR', 'cross_street': '', 'formatted_address': 'St. Bancario Norte 1 - Asa Norte, Brasília, DF', 'locality': 'Brasília', 'region': 'DF'}, 'name': 'Downtown Restaurante Escola SENAC', 'price': 2, 'rating': 8.4, 'stats': {'total_photos': 4, 'total_ratings': 34, 'total_tips': 4}}, {'fsq_id': '58414c334bafb71470eb004b', 'categories': [{'id': 13046, 'name': 'Ice Cream Parlor', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_', 'suffix': '.png'}}, {'id': 13065, 'name': 'Restaurant

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [7]:
def parse_response(response):
    data_fq = response.json()

    if 'results' in data_fq:
        pois = []

        for result in data_fq['results']:
            fsq_id = result['fsq_id']
            name = result['name']
            rating = result.get('rating', None)
            categories = [category['name'] for category in result.get('categories', [])]
            location = result.get('location', {})
            address = location.get('formatted_address', '')
            latitude = location.get('latitude', '')
            longitude = location.get('longitude', '')

            poi = {
                'fsq_id': fsq_id,
                'name': name,
                'rating': rating,
                'categories': categories,
                'address': address,
                'latitude': latitude,
                'longitude': longitude
            }

            # Filter by restaurant category
            if 'Restaurant' in categories:
                pois.append(poi)

        return pois

    return []

# Create an empty list to store the parsed results
parsed_results = []

# Iterate over each row in the DataFrame
for _, row in df.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    url = f"https://api.foursquare.com/v3/places/search?ll={latitude}%2C{longitude}&radius=1000&fields=categories%2Crating%2Cgeocodes%2Ccategories%2Clocation%2Cname%2Cfsq_id%2Cstats%2Cprice&limit=50"

    headers = {
        "accept": "application/json",
        "Authorization": foursquare_key
    }
    
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    pois = parse_response(response)

    # Extend the parsed results list with the new POIs
    parsed_results.extend(pois)

Put your parsed results into a DataFrame

In [9]:
# Create a DataFrame from the parsed results
df_foursquare = pd.DataFrame(parsed_results)

# Display the parsed DataFrame
print(df_foursquare)

                       fsq_id                                          name  \
0    4cfe88e1084f548186099109                           Torteria Di Lorenza   
1    548b5911498ecccac953fc61                                      Café Oyá   
2    537f6252498e54e075c1b714                               Dulce Patagonia   
3    4e97331df5b91722c72e3cf2                               Delícia de Bolo   
4    5682ee79498e148ae235ced0  Philippe Verstraete Pâtisserie e Boulangerie   
..                        ...                                           ...   
385  5706d023498eb7eb702acffe                                    Gádio Café   
386  4bb3624a2397b713070338b3                                    Kopenhagen   
387  53d6cffd498ee7e6da02f4df                            Chiquinho Sorvetes   
388  4ca75e32a6e08cfa4dd08394                                     Yogoberry   
389  515cceace4b0c49b3f20da69                                   Malbec Beer   

     rating                                        

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [62]:

def get_yelp(latitude, longitude, yelp_key):
    url = "https://api.yelp.com/v3/businesses/search"

    headers = {
        "Authorization": f"Bearer {yelp_key}"
    }

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "radius": 1000
    }

    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()

    data_yelp = response.json()

    if 'businesses' in data_yelp:
        return data_yelp['businesses']
    else:
        return []

# Call the function for each bike station
yelp_results = []

for _, row in df.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']

    businesses = get_yelp(latitude, longitude, yelp_key)
    yelp_results.extend(businesses)

# Process the Yelp results as needed
for business in yelp_results:
    # Extract the details you want (name, rating, location, etc.)
    name = business['name']
    rating = business.get('rating', None)
    location = business.get('location', {})
    address = location.get('address1', '')
    city = location.get('city', '')
    state = location.get('state', '')
    zip_code = location.get('zip_code', '')

    # Perform further processing or store the details as needed
    print(f"Name: {name}")
    print(f"Rating: {rating}")
    print(f"Address: {address}")
    print(f"City: {city}")
    print(f"State: {state}")
    print(f"ZIP Code: {zip_code}")
    print("------------------------")


Name: Maori Gastronomia
Rating: 4.5
Address: CLN, 110
City: Brasília
State: DF
ZIP Code: 70753-000
------------------------
Name: Feitiço Mineiro Restaurante
Rating: 4.5
Address: CLN 306 BL D  lj 45/51
City: Brasília
State: DF
ZIP Code: 70745-540
------------------------
Name: Sushi Woman
Rating: 4.0
Address: CLN 109 BL A  lj 20
City: Brasília
State: DF
ZIP Code: 70752-510
------------------------
Name: Mucho Gusto Gastronomia
Rating: 4.5
Address: CLN 309 BL A  lj 20
City: Brasília
State: DF
ZIP Code: 70755-510
------------------------
Name: BFC
Rating: 4.0
Address: Cln 110 BLOCO B LOJA 62
City: Shcn
State: DF
ZIP Code: 
------------------------
Name: Asa Gaúcha Restaurante
Rating: 4.5
Address: SCLRN 709 BL E, 56
City: Brasília
State: DF
ZIP Code: 70750-515
------------------------
Name: El Paso Texas
Rating: 4.0
Address: CLN 110
City: Brasília
State: DF
ZIP Code: 70753-520
------------------------
Name: Kikebab Kebaberia
Rating: 3.5
Address: SCLN 110 Bloco B, 38
City: Brasília
State: 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [67]:
def get_yelp_results(latitude, longitude, yelp_key):
    url = "https://api.yelp.com/v3/businesses/search"

    headers = {
        "Authorization": f"Bearer {yelp_key}"
    }

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "radius": 1000,
        "categories": "restaurants"
    }

    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()

    data_yelp = response.json()

    if 'businesses' in data_yelp:
        return data_yelp['businesses']
    else:
        return []

# Call the function for each bike station
yelp_results = []

for _, row in df.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']

    businesses = get_yelp_results(latitude, longitude, yelp_key)
    yelp_results.extend(businesses)

# Process the Yelp results as needed
for business in yelp_results:
    # Extract the details you want (name, rating, location, etc.)
    name = business['name']
    rating = business.get('rating', None)
    location = business.get('location', {})
    address = location.get('address1', '')
    city = location.get('city', '')
    state = location.get('state', '')
    zip_code = location.get('zip_code', '')

    # Perform further processing or store the details as needed
    print(f"Name: {name}")
    print(f"Rating: {rating}")
    print(f"Address: {address}")
    print(f"City: {city}")
    print(f"State: {state}")
    print(f"ZIP Code: {zip_code}")
    print("------------------------")


Name: Maori Gastronomia
Rating: 4.5
Address: CLN, 110
City: Brasília
State: DF
ZIP Code: 70753-000
------------------------
Name: Feitiço Mineiro Restaurante
Rating: 4.5
Address: CLN 306 BL D  lj 45/51
City: Brasília
State: DF
ZIP Code: 70745-540
------------------------
Name: Sushi Woman
Rating: 4.0
Address: CLN 109 BL A  lj 20
City: Brasília
State: DF
ZIP Code: 70752-510
------------------------
Name: Mucho Gusto Gastronomia
Rating: 4.5
Address: CLN 309 BL A  lj 20
City: Brasília
State: DF
ZIP Code: 70755-510
------------------------
Name: BFC
Rating: 4.0
Address: Cln 110 BLOCO B LOJA 62
City: Shcn
State: DF
ZIP Code: 
------------------------
Name: Asa Gaúcha Restaurante
Rating: 4.5
Address: SCLRN 709 BL E, 56
City: Brasília
State: DF
ZIP Code: 70750-515
------------------------
Name: El Paso Texas
Rating: 4.0
Address: CLN 110
City: Brasília
State: DF
ZIP Code: 70753-520
------------------------
Name: Kikebab Kebaberia
Rating: 3.5
Address: SCLN 110 Bloco B, 38
City: Brasília
State: 

Put your parsed results into a DataFrame

In [73]:
# Create a list to store the parsed results
parsed_results = []

# Process the Yelp results and store them in the list
for business in yelp_results:
    # Extract the details you want (name, rating, location, etc.)
    name = business['name']
    rating = business.get('rating', None)
    location = business.get('location', {})
    address = location.get('address1', '')
    city = location.get('city', '')
    state = location.get('state', '')
    zip_code = location.get('zip_code', '')

    # Create a dictionary for the parsed result
    parsed_result = {
        'Name': name,
        'Rating': rating,
        'Address': address,
        'City': city,
        'State': state,
        'ZIP Code': zip_code
    }

    # Append the parsed result to the list
    parsed_results.append(parsed_result)

# Create a DataFrame from the parsed results
df_yelp= pd.DataFrame(parsed_results)

# Print the DataFrame
print(df_yelp)


                                                   Name  Rating  \
0                                     Maori Gastronomia     4.5   
1                           Feitiço Mineiro Restaurante     4.5   
2                                           Sushi Woman     4.0   
3                               Mucho Gusto Gastronomia     4.5   
4                                                   BFC     4.0   
...                                                 ...     ...   
1265                     Sabor Glace Restaurante e Café     4.0   
1266                                     Risotti D Ouro     5.0   
1267                                         Diamantina     5.0   
1268                                              Bob's     5.0   
1269  Restaurante Associação dos Empregados Appex Br...     4.0   

                      Address      City State   ZIP Code  
0                    CLN, 110  Brasília    DF  70753-000  
1      CLN 306 BL D  lj 45/51  Brasília    DF  70745-540  
2         CLN 109 

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

In [118]:
# Count the number of results
foursquare_count = len(df_foursquare)
yelp_count = len(df_yelp)

print("Number of results:")
print(f"Foursquare: {foursquare_count}")
print(f"Yelp: {yelp_count}")



Number of results:
Foursquare: 390
Yelp: 1270


In [119]:
# Compare the availability of desired information
foursquare_complete = df_foursquare[['name', 'rating', 'address']].dropna()
yelp_complete = df_yelp[['Name', 'Rating', 'Address']].dropna()

foursquare_completeness = len(foursquare_complete) / foursquare_count
yelp_completeness = len(yelp_complete) / yelp_count

print("Completeness of desired information:")
print(f"Foursquare: {foursquare_completeness:.2%}")
print(f"Yelp: {yelp_completeness:.2%}")


Completeness of desired information:
Foursquare: 100.00%
Yelp: 100.00%


Based on the provided results, here's the comparison:

Foursquare returned 390 results, and out of those, 94.62% have the desired information (name, rating, and address) available.
Yelp returned 1270 results, and all of them (100%) have the desired information available.

Get the top 10 restaurants according to their rating

In [90]:
# Remove duplicates based on 'name' column
df_foursquare_unique = df_foursquare.drop_duplicates(subset='name')

# Sort the DataFrame by rating in descending order
top_10_restaurants_foursquare = df_foursquare_unique.sort_values(by='rating', ascending=False).head(10)

# Print the top 10 restaurants
print(top_10_restaurants_foursquare[['name', 'rating', 'address']])


                                            name  rating  \
316  Bacio di Latte - Shopping Conjunto Nacional     9.1   
50                                       Oni-Uno     9.1   
71                          La Petit Boulangerie     9.0   
2                                Dulce Patagonia     8.9   
87                   Cachorro Quente do Edivaldo     8.8   
185                          Confeitaria Mineira     8.8   
289                      Castália Padaria e Café     8.7   
123                             Bischoff Gourmet     8.7   
48                               Blend Boucherie     8.6   
45                                      Santé 13     8.6   

                                               address  
316  SDN Conjunto A Shopping Conjunto Nacional, Bra...  
50   Quadra Clsw 103 Block B Local 1 subsolo (Ed. R...  
71                               Cln 212, Brasília, DF  
2                Cln 309 Bl C, Brasília, DF, 70755-530  
87                   Sqn, 104, Brasília, DF, 70733-000

In [88]:
# Sort the DataFrame by rating in descending order
top_10_restaurants = df_yelp.sort_values(by='Rating', ascending=False).head(10)

# Print the top 10 restaurants
print(top_10_restaurants[['Name', 'Rating', 'Address']])


                          Name  Rating  \
733              Peixe na Rede     5.0   
1125                 Coxa Cone     5.0   
1149              Espaço Grill     5.0   
1147                     Bob's     5.0   
542   Restaurante Fogão Nativo     5.0   
539                    Hezerta     5.0   
536               TNT Fit Food     5.0   
533                   Maki San     5.0   
530                      Meatz     5.0   
1135             Rural Mansões     5.0   

                                       Address  
733                                    CLS 405  
1125                Shopping Conjunto Nacional  
1149                      SCN Q 2 BL A  lj 300  
1147                      CC Conjunto Nacional  
542                                     SIG, 3  
539                       CLSW 101 Bloco C, 75  
536   Primeira Avenida, 5 - Sudoeste, Cruzeiro  
533                                   CLSW 301  
530                                      QI 27  
1135                   R. da Escola do Córrego 

In [91]:
# Export the DataFrame to a CSV file
df_foursquare.to_csv('df_foursquare.csv', index=False)
df_yelp.to_csv('df_yelp.csv', index=False)


In [97]:
# Scale the rating column in df_foursquare
max_rating_foursquare = df_foursquare['rating'].max()
df_foursquare['rating'] = (df_foursquare['rating'] / max_rating_foursquare) * 5


In [126]:
merged_df = pd.merge(df_foursquare, df_yelp, on=['latitude', 'longitude'], how='inner')


KeyError: 'latitude'

In [114]:
top_10 = merged_df.nlargest(10, 'rating')
top_10_unique = top_10.drop_duplicates(subset=['Name', 'Rating', 'Address'])
display(top_10_unique[['Name', 'Rating', 'Address']])



,Name,Rating,Address
0,Beirute,5,CLS 109
4,Panificadora Savassi,5,"CLN 307 Bloco E, 49"
10,Fran's Café,5,"CLN 209 Bloco D, 17"
58,Pinella,5,CLN 408
98,Cannelle,5,"CLN, 213"
100,C'est si Bon,5,CLN 213
101,Santé 13,5,"SCLN 413, Bloco A, 40"


In [108]:
merged_df.to_csv('merged_data.csv', index=False)


In [128]:
df_yelp

,Name,Rating,Address,City,State,ZIP Code
0,Maori Gastronomia,5,"CLN, 110",Brasília,DF,70753-000
1,Feitiço Mineiro Restaurante,5,CLN 306 BL D lj 45/51,Brasília,DF,70745-540
2,Sushi Woman,5,CLN 109 BL A lj 20,Brasília,DF,70752-510
3,Mucho Gusto Gastronomia,5,CLN 309 BL A lj 20,Brasília,DF,70755-510
4,BFC,5,Cln 110 BLOCO B LOJA 62,Shcn,DF,
...,...,...,...,...,...,...
1265,Sabor Glace Restaurante e Café,5,SCN Q 2 BL A lj 5,Brasília,DF,70712-900
1266,Risotti D Ouro,5,SCN Q 5 BL A lj 226C,Brasília,DF,70715-900
1267,Diamantina,5,"SHN Quadra 2, Bloco E",Brasília,DF,70702-904
1268,Bob's,5,CC Conjunto Nacional,Brasília,DF,70077-000
